### Building RAG System with LangChain and FAISS

In [1]:
#importing libraries

import os
from dotenv import load_dotenv
load_dotenv()

#langchain imports

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

d:\RAGs In Depth\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#data ingestion and documents

sample_documents=[
    Document(page_content='''
Machine learning is a branch of artificial intelligence that enables computers to learn from data and improve performance without being explicitly programmed. 
It works by feeding large datasets into algorithms that identify patterns and relationships, which can then be used to make predictions or decisions on new, unseen data. 
Unlike traditional programming, where rules must be manually defined, machine learning systems adapt and evolve as they encounter more information. 
This makes them especially powerful for complex tasks such as image recognition, speech processing, natural language translation, fraud detection, and personalized recommendations. 
By leveraging techniques like supervised learning, unsupervised learning, and reinforcement learning, machine learning has become a cornerstone of modern technology, powering applications from healthcare diagnostics to self-driving cars
''', 
metadata={"source": "Machine Learning", "page": 1, "topic": "AI"}),

Document(page_content='''
        Deep learning is a specialized branch of machine learning that uses multilayered neural networks to process and learn from complex data. It mimics the way the human brain works by stacking artificial neurons into layers, allowing computers to automatically discover intricate patterns and representations without manual feature engineering. The term �deep� refers to the use of many hidden layers�sometimes hundreds or thousands�that transform input data step by step until a meaningful output is produced. These architectures can be trained in supervised, semi-supervised, or unsupervised ways, making them highly versatile.
Deep learning has revolutionized fields such as computer vision, natural language processing, speech recognition, and autonomous systems. For example, convolutional neural networks (CNNs) excel at image classification, while recurrent and transformer-based networks dominate language tasks. By leveraging vast datasets and powerful GPUs, deep learning models achieve state-of-the-art performance in tasks ranging from facial recognition to self-driving cars. Its ability to handle unstructured data�like images, audio, and text�sets it apart from traditional machine learning approaches, making deep learning a cornerstone of modern artificial intelligence )
''',
metadata={"source": "Deep Learning", "page": 1, "topic": "AI"}),

Document(page_content='''
         Natural Language Processing (NLP) is a field of artificial intelligence that focuses on enabling computers to understand, interpret, and generate human language. It combines linguistics, computer science, and machine learning to bridge the gap between human communication and digital systems. NLP techniques allow machines to process text and speech, making it possible to perform tasks such as sentiment analysis, language translation, speech recognition, chatbots, and information retrieval. By breaking down language into components like syntax, semantics, and context, NLP systems can extract meaning and respond intelligently. Modern NLP relies heavily on deep learning models, such as transformers, which power applications like voice assistants, automated customer support, and large-scale text analysis. This technology is central to how humans interact naturally with machines today, making it one of the most impactful areas of AI.
''',
metadata={"source": "Natural Language Processing", "page": 1, "topic": "AI"})
]

sample_documents

[Document(metadata={'source': 'Machine Learning', 'page': 1, 'topic': 'AI'}, page_content='\nMachine learning is a branch of artificial intelligence that enables computers to learn from data and improve performance without being explicitly programmed. \nIt works by feeding large datasets into algorithms that identify patterns and relationships, which can then be used to make predictions or decisions on new, unseen data. \nUnlike traditional programming, where rules must be manually defined, machine learning systems adapt and evolve as they encounter more information. \nThis makes them especially powerful for complex tasks such as image recognition, speech processing, natural language translation, fraud detection, and personalized recommendations. \nBy leveraging techniques like supervised learning, unsupervised learning, and reinforcement learning, machine learning has become a cornerstone of modern technology, powering applications from healthcare diagnostics to self-driving cars\n'),

In [3]:
#text splitting

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "]
)

chunks = text_splitter.split_documents(sample_documents)
chunks

[Document(metadata={'source': 'Machine Learning', 'page': 1, 'topic': 'AI'}, page_content='Machine learning is a branch of artificial intelligence that enables computers to learn from data and improve performance without being explicitly programmed. \nIt works by feeding large datasets into algorithms that identify patterns and relationships, which can then be used to make predictions or decisions on new, unseen data. \nUnlike traditional programming, where rules must be manually defined, machine learning systems adapt and evolve as they encounter more information. \nThis makes them'),
 Document(metadata={'source': 'Machine Learning', 'page': 1, 'topic': 'AI'}, page_content='they encounter more information. \nThis makes them especially powerful for complex tasks such as image recognition, speech processing, natural language translation, fraud detection, and personalized recommendations. \nBy leveraging techniques like supervised learning, unsupervised learning, and reinforcement learni

In [4]:
print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 8 chunks from 3 documents

Example chunk:
Content: Machine learning is a branch of artificial intelligence that enables computers to learn from data and improve performance without being explicitly programmed. 
It works by feeding large datasets into algorithms that identify patterns and relationships, which can then be used to make predictions or decisions on new, unseen data. 
Unlike traditional programming, where rules must be manually defined, machine learning systems adapt and evolve as they encounter more information. 
This makes them
Metadata: {'source': 'Machine Learning', 'page': 1, 'topic': 'AI'}


In [5]:
#embeddings
embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
#vectorstore and retrievers

vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

print(f"vectorstore created with {vectorstore.index.ntotal} vectors")

vectorstore created with 8 vectors


In [7]:
#save vectorstore in local machine

vectorstore.save_local("faiss_index")
print("vectorstore saved in 'faiss_index' directory")

vectorstore saved in 'faiss_index' directory


In [8]:
#to load vectstore from local machine

faiss_store = FAISS.load_local(
    folder_path="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

In [9]:
#similarity search

query="what is NLP?"
results = faiss_store.similarity_search(query, k=2)
results

[Document(id='7ba2b890-01bc-4aae-8f82-7add5df85455', metadata={'source': 'Natural Language Processing', 'page': 1, 'topic': 'AI'}, page_content='Natural Language Processing (NLP) is a field of artificial intelligence that focuses on enabling computers to understand, interpret, and generate human language. It combines linguistics, computer science, and machine learning to bridge the gap between human communication and digital systems. NLP techniques allow machines to process text and speech, making it possible to perform tasks such as sentiment analysis, language translation, speech recognition, chatbots, and information'),
 Document(id='9d1ee29e-abef-4db7-a4ff-604c61ac9fa2', metadata={'source': 'Natural Language Processing', 'page': 1, 'topic': 'AI'}, page_content='speech recognition, chatbots, and information retrieval. By breaking down language into components like syntax, semantics, and context, NLP systems can extract meaning and respond intelligently. Modern NLP relies heavily on 

In [10]:
print(f"Query: {query}")
print("Top 2 similar chunks")
for i, doc in enumerate(results):
    print(f"\n{i+1}. Source: {doc.metadata['source']}")
    print(f"  Content: {doc.page_content[:200]}...")

Query: what is NLP?
Top 2 similar chunks

1. Source: Natural Language Processing
  Content: Natural Language Processing (NLP) is a field of artificial intelligence that focuses on enabling computers to understand, interpret, and generate human language. It combines linguistics, computer scie...

2. Source: Natural Language Processing
  Content: speech recognition, chatbots, and information retrieval. By breaking down language into components like syntax, semantics, and context, NLP systems can extract meaning and respond intelligently. Moder...


In [11]:
#search wth meta data filtering

filter_dict={"source":"Deep Learning"}
filtered_results= faiss_store.similarity_search(
    query,
    k=2,
    filter=filter_dict
)

filtered_results

[Document(id='30ea8fb7-a012-484c-afa9-054d1d9d2c53', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'AI'}, page_content='input data step by step until a meaningful output is produced. These architectures can be trained in supervised, semi-supervised, or unsupervised ways, making them highly versatile.\nDeep learning has revolutionized fields such as computer vision, natural language processing, speech recognition, and autonomous systems. For example, convolutional neural networks (CNNs) excel at image classification, while recurrent and transformer-based networks dominate language tasks. By leveraging vast'),
 Document(id='7d41daf7-3e03-4b11-84b8-a7f77cf00a9a', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'AI'}, page_content='Deep learning is a specialized branch of machine learning that uses multilayered neural networks to process and learn from complex data. It mimics the way the human brain works by stacking artificial neurons into layers, allowing computers to

In [12]:
#building RAG chain with LCEL

from langchain.chat_models import init_chat_model
model = init_chat_model(
    model="groq:llama-3.1-8b-instant"
)

model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000018C0A687C10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018C0A7E1450>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
#prompt template

prompt = ChatPromptTemplate.from_template('''
    You are a question answering chatbot, answer given question only using context provided.
    if question cannot be answered from given context, please say I don't know.
    context: {context}
    question: {input} ''')

In [14]:
#retriever
retriever = faiss_store.as_retriever()

In [ ]:
from typing import List

#format documents for a prompt
def format_docs(docs:List[Document])->str:
    "format documents for insertion into prompt"
    formatted=[]
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'unknown')
        formatted.append(f"Document {i+1} (source: {source}): \n{doc.page_content}")
    return "\n\n".join(formatted) 
   
    

In [16]:
simple_rag_chain =(
    {"context": retriever | RunnableLambda(format_docs),
     "question": RunnablePassthrough()}
     | prompt
     | model
     | StrOutputParser()
) 

simple_rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018C08C347D0>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="\n    You are a question answering chatbot, answer given question only using context provided.\n    if question cannot be answered from given context, please say I don't know.\n    context: {context}\n    question: {input} "), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'too

In [17]:
#Conversationl Rag Chain

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant, Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("user", "Context: {context}\n\n Question: {input}")
])

conversational_prompt

ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

In [18]:
def create_conversational_rag():
    "create a conversational rag chain with memory"
    return(
        RunnablePassthrough.assign(
            context=lambda x : format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | model
        | StrOutputParser()
    )

conversational_rag=create_conversational_rag()

conversational_rag

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_docs(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag=

In [19]:
#streaming rag chain
streaming_rag_chain = (
    {"context": retriever | format_docs,
     "question": RunnablePassthrough()}
    | prompt
    | model 
)

streaming_rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018C08C347D0>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="\n    You are a question answering chatbot, answer given question only using context provided.\n    if question cannot be answered from given context, please say I don't know.\n    context: {context}\n    question: {input} "), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'too

In [20]:
# test function for different chain types

def test_rag_chains(question:str):
    "Test all RAG chain variants"
    print(f"Question: {question}")
    print("=" * 60)

    #1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke({"input":question})
    print(f"Answer: {answer}")


In [21]:
test_rag_chains("what is ml?")

Question: what is ml?

1. Simple RAG Chain:


AttributeError: 'dict' object has no attribute 'replace'